In [1]:
import os.path
import pandas as pd

import fitz #PyMuPDF

import string
translation_table = str.maketrans('', '', string.digits)

import PyPDF2

from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH, Language.CHINESE, Language.TAGALOG, Language.SOTHO, Language.YORUBA]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
#detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()

In [2]:
if (os.path.exists("data/langmeta.csv")):
    os.remove("data/langmeta.csv")

In [3]:
pdfurl = pd.read_csv("data/downloadedpdf.csv").fillna(value = "")

In [4]:
text= ""
for index, row in pdfurl.iterrows():    
    unique = row['URL'].removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
    unique = unique.removesuffix("\n")
    metadict = {"Identifier":unique}   
    pfad = "temp/" + unique + ".pdf"
#Falls beim Download kein Error aufgetreten ist: Probier die Datei zu öffnen
    if((row['Error'] == "")):
        #saved as .txt if .txt doesn't exist yet
        textpfad = "temp/" + unique + ".txt"
        if (not os.path.exists(textpfad)):
            try:
                pdfFileObj = open(pfad, 'rb')
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                for x in range(0,pdfReader.numPages):        
                    pageObj = pdfReader.getPage(x)        
                    text = text + pageObj.extractText()    
            except:
                print("An Error occured while reading.")
                #metadict.update({"Error":'OpeningError'})
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/langmeta.csv")):   
                    #oldframe = pd.read_csv("data/langmeta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/langmeta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/langmeta.csv", index=False)
                    oldframe = frame
                metadict.clear
                continue
            #save
            with open(textpfad, 'w', encoding="utf-8") as textfile:
                textfile.write(text.encode('utf-8', 'replace').decode())
        else:
            with open(textpfad, 'r', encoding="utf-8") as textfile:
                text = textfile.read()
    #Größe bestimmen
        metadict.update({"Textsize":len(text)})
    #Titel bestimmen
        with fitz.open(pfad) as pdffile:
            header = ""  
            header_size = 0.0
            first_span = True  
   
            for page in pdffile:
                txtpage = page.get_textpage()
                DICT = txtpage.extractDICT()
                for block in DICT["blocks"]:
                    if block['type'] == 0:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                if(first_span == True):
                                    header = span["text"]
                                    header_size = span["size"]
                                    first_span = False
                                elif(span["size"] > header_size):
                                    header = span["text"]
                                    header_size = span["size"]    
                break
            metadict.update({"texttitle":header})
    #Headerextraction
        headertextpfad = "temp/" + unique + "_header.txt"
        if (not os.path.exists(headertextpfad)):
        #Die verschiedenen Schiftgrößen bestimmen
            with fitz.open(pfad) as pdffile:
                sizedict = {0:0}
                fullsize = 0
                for page in pdffile:
                    txtpage = page.get_textpage()
                    DICT = txtpage.extractDICT()
                    for block in DICT["blocks"]:
                        if block['type'] == 0:
                            for line in block["lines"]:
                                for span in line["spans"]:
                                    if(int(span["size"]) in sizedict):
                                        sizedict[int(span["size"])] += len(span["text"])
                                    else:
                                        sizedict[int(span["size"])]= len(span["text"])
                                    fullsize += len(span["text"])
                                
            #Welche Größen sind Header?
            if(fullsize > 0):
                headersize = 0
                sortsizedict = sorted(sizedict, reverse=True)
                i_max = 0
                for key in sortsizedict:
                    i = i_max -1
                    if(i>=0):
                        sizedict[sortsizedict[i_max]]+= sizedict[sortsizedict[i]]
                    if(sizedict[sortsizedict[i_max]] > fullsize/4):
                        i_max -= 1
                        break
                    i_max += 1
            
                if(i_max < 0):
                    i_max = 0
                headersize = sortsizedict[i_max]
        
            #header auslesen
                headertext=""
                with fitz.open(pfad) as pdffile:
                    for page in pdffile:
                        txtpage = page.get_textpage()
                        DICT = txtpage.extractDICT()
                        for block in DICT["blocks"]:
                            if block['type'] == 0:
                                for line in block["lines"]:
                                    for span in line["spans"]:
                                        if(int(span["size"]) >= headersize):
                                            headertext += span["text"].translate(translation_table) +" "
                with open(headertextpfad, 'w', encoding="utf-8") as textfile:
                    textfile.write(headertext.encode('utf-8', 'replace').decode())   
                                
    #Spracherkennung
        if(len(text)>=100):            
            language = detector.detect_language_of(text) #compute_language_confidence_values(text)
            if(language is not None):
                metadict.update({"Language":language.name})
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/langmeta.csv")):   
            #oldframe = pd.read_csv("data/langmeta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/langmeta.csv", index=False)
            oldframe = newframe
        else:
            frame.to_csv("data/langmeta.csv", index=False)
            oldframe = frame
        metadict.clear
        text=""
        print("Finished: ", unique)

Finished:  8I6sM5zapD60
Finished:  8ZICOHBmAHyQ
Finished:  3ztCv-WpxJ4U
Finished:  6mOhjfscZK2A
Finished:  1eteONeHL82Y
Finished:  1mjbqKfwSW7U
Finished:  1BruMQFjEIRY
Finished:  1hZf0CFwsBhA
Finished:  1PrtXFruGers
Finished:  1LvS8_gxjyYc
Finished:  18r_JtdoG2RA
Finished:  1zfzKOhH3Szw
Finished:  1LtI4gnpDFIQ
Finished:  17pFIynqIjGc
Finished:  1Xamc2rgc3UA
Finished:  1055-D7j6GpXg
Finished:  9kfa9dR1C5xw
Finished:  1lWB4XHk4REw
Finished:  1R6Y6mMha5-E
Finished:  11r0NIcRyxxo
Finished:  19CzcUUPmFeI
Finished:  1_RecnQsU3os
Finished:  1CnMqavsnGA0
Finished:  1fCUEOJoo3Hg
Finished:  13_l81m9RJSc
Finished:  1igaErcLDtkc
Finished:  1NiNIFtAGLOg
Finished:  11yrO9UTRBw4
Finished:  1hpsAD9zjv90
Finished:  1nmWdGNZ7Iuc
Finished:  1yFnYVkqvY1U
Finished:  1d0nPftZxHxc
Finished:  11CQsDeYsCBI
Finished:  1WZsTPgDjvsA
Finished:  1tMPU2R0LTM8
Finished:  1aWRhGTYtlxs
Finished:  1nSyVPO2adeI
Finished:  10Zs2rFMBxEI
Finished:  11FKz-hA2FGSE
Finished:  16dS9yDCfuPMQ
Finished:  6Z4w4nrMk_dA
Finished:  4u

Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished:  4BqncnRS-xuQ
An Error occured while reading.
Finished:  7DiDfT97sT5A
Finished:  7sykWlrvqq-o
Finished:  7OS7rgsjiOtI
Finished:  71ZhNVEZLC_Y
Finished:  7i00Q-WpSkLg
Finished:  48i6Hb5hrRw8
Finished:  9ouqCkhhlSCs
An Error occured while reading.
Finished:  3MZNl2KRCMXo
Finished:  9j3OADe68S1E
Finished:  7rkjI9ssjrhE
Finished:  74Hd90cgceek
Finished:  9LJ3o9jk3tec
Finished:  3meMN8XI4a1I
Finished:  9hz9f6wzzUk8
Finished:  8394JVb2Sy_E
Finished:  6I-lYeH7oRIg
Finished:  3cMAp8O1PDjk
Finished:  4Go34OQK9Xus
Finished:  8NU3_-wc8J6U
Finished:  70yaDUVKkzO0
Finished:  80T7TVMBvgtA
Finished:  12Pt-Pn39TrU8
Finished:  5oTD8FZELP-A
Finished:  12in73t84Dil0
Finished:  3i0XxrLwT-B0
Finished:  3uSefZ10MMRI
Finished:  31QgCtWiHPwk
Finished:  3RW9AC2a0poU
Finished:  3Q_jRQK7qx3Q
Finished:  3T7n2K__oj-Y
Finished:  73wEHU-Q9s9o
Finished:  5PW8cXlnZMlM
Finished:  3VjjJ2ztm6xY
Finished:  6qxb8dHzqdQQ
Finished:  6s-mf2mhmZLU
Finished:  55z15y35ifP0
Finished:  1EJTvwygeiSY
Finished:  7ejSdpUodoE

Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished:  4HHbdSBFoPig
An Error occured while reading.
Finished:  3tly8z01eTsk
Finished:  4clkJ5Vby39s
Finished:  4EbE0xuYrslo
Finished:  9-5x0ORImi7s
Finished:  3k9EYQicZjwY
Finished:  7WYgaHv41V_w
Finished:  5lzay-4HbiuI
Finished:  6NfuX2Pp_uIo
Finished:  3N2nfOXLTjjQ
Finished:  451PeKx69zFQ
Finished:  46WNGD89SKY4
Finished:  6hzSWps-w1nk
Finished:  723L8Ft5Ms80
Finished:  5gQyopFTbD8I
Finished:  4hpWUwBqq-X8
Finished:  4RwsNYcbo7as
Finished:  4-3T0ElKv3bI
Finished:  4DxsWMGukBVk
Finished:  3v_Ju1xli2oE
Finished:  5rX-bl0dg2n8
An Error occured while reading.
Finished:  9F2rEnW9U2W0
Finished:  3kG_0NOP8Gfs
Finished:  740btzv3L2O0
Finished:  3k3IBMrDlAFg
Finished:  3FZhWmM6Zwbc
Finished:  43Kq3-1Ku-1A
Finished:  9YHqr0iRrTek
Finished:  9_vlQ7Q3vLQE
Finished:  3jsbZDIkI57E
Finished:  15T55Qup56oi8
Finished:  9lPXAR2YyDyQ
Finished:  5akt7Zo_5d6Y
Finished:  6qyTSU4wZaTE
Finished:  1466C3XzOm2-w
Finished:  65KDgPzNvrWc
Finished:  5sBXCeARg-bI
Finished:  99TopKFWKIMo
Finished:  8pt4IAdDXJ4

...

Finished:  76_jpFfzixG4
Finished:  7-azt_sjmU6M
Finished:  7UogJJUn5oG0
Finished:  8TEzAOtd-IuA
An Error occured while reading.
Finished:  8yb2cSIM0JoM
Finished:  8nqTM1F9OZQY
Finished:  8UM2GB1R_YEM
Finished:  8SQuV8OYCMaM
Finished:  8CX835BHq1cg
Finished:  8dbEWgiAOHD4
Finished:  8IJUu_tV7HXM
Finished:  8NWw3ODNyMNI
Finished:  8QULKSqjR_hs
Finished:  8rKaSV3s1xI8
Finished:  8Kq2QYcfHYAo
Finished:  8MyjRtDC5FtQ
Finished:  8EwHTtH6KU9E
Finished:  8JkE6o4sG5IQ
Finished:  7TACti3nPqy4
Finished:  8GjRm3PdQglE
Finished:  8JN7WxH-eoE8
Finished:  8D1B00DoLcxE
Finished:  8VpTNhXmu_IY
Finished:  8BHgZrc-BLjg
Finished:  11ArcjoCFvfiQ
Finished:  3qUOy4iAseWU
Finished:  814or6gqvIfE
Finished:  5X-cE-KGPhLY
Finished:  9DGmZ0LAlRek
Finished:  11ZNg3MSPAR4g
An Error occured while reading.
Finished:  7Hz6jHSGno6Y
Finished:  7sVpPhwd35RM
Finished:  9adyfkfdOX80
Finished:  9FxEMBYWwbQg
Finished:  4UrV4f05Ri2A
Finished:  5SRnPKV3MmCw
Finished:  48m3EEeCoO6c
Finished:  11QOjoVjBNrek
Finished:  93N5g5iqS_

Finished:  6NmgBWRVYp54
Finished:  6iB0i5lbHYLk
Finished:  6kAvSjCKl9s8
Finished:  6XfouMnJ1kS8
Finished:  6xn8fpTuTu_c
Finished:  6Vm_sMKKyZ5g
Finished:  61walY_bjmEA
Finished:  6lhn_6FYMO4c
Finished:  6SbfdtjSJ9ZQ
Finished:  6uq9Arwq8_y4
Finished:  67fusDBlMa0s
Finished:  6x1rxYCExG44
Finished:  6OTBIAUHnViU
Finished:  6Y8YCU_QR_oE
Finished:  6SQoBBiliQrA
Finished:  6ZdTCOeyfPO4
Finished:  62DvjL3kxbq0
Finished:  6q-uITUuM1xU
Finished:  6-wNK04DdfzQ
Finished:  6PqSnZq0ncvw
Finished:  6V7KFoK1crfM
Finished:  6z-gCAbuWOto
Finished:  6MIY5YgNoRBA
Finished:  6V-VuIceteyM
Finished:  6bJhAciKkdwo
Finished:  6ZV5Hd4MwS1A
Finished:  6oyDCJ37wJmA
Finished:  6xT2Rf3JDgB0
Finished:  6Y0NebTRraLU
Finished:  6YFfcrnY_ICI
Finished:  6AsCuVld3XIk
Finished:  6ZgPRd5VLnss
Finished:  6qeYwLvOB5VU
Finished:  6SqVnFKU4Imk
Finished:  64qnscMLeFAM
Finished:  6expinySE_K8
Finished:  6T2YdWynYgPc
Finished:  6Mi5V0pSAALM
Finished:  6AyvquExwcdc
Finished:  6PbNqw9J4yrA
Finished:  6fnS-5ah732c
Finished:  6Z84r

Finished:  11DkJZZ1scNjI
Finished:  11XGIm65r9r8c
Finished:  81Q_Ai3OIQ7o
Finished:  12oEw9-aEbvwM
Finished:  1237YBKoYyxgw
Finished:  120P2pFLjBfXg
Finished:  12IBEs76n95Ek
Finished:  120LGX7bIParM
Finished:  121qFmEuuAWQY
Finished:  127wDDblYRlNQ
Finished:  11axDTkQDG4kw
An Error occured while reading.
Finished:  11i1Vn_n0B4OE
Finished:  11DwbyxtY6AO0
An Error occured while reading.
Finished:  11MZWZxjZNyrw
An Error occured while reading.
An Error occured while reading.
Finished:  1143eQF1qpymI
Finished:  11H8gOBeToCzw
Finished:  11TXv-hhS4WRM
Finished:  11r2kRYV7S7V4
Finished:  11slcUMDM45vE
Finished:  11P1AcsabYDzQ
Finished:  11vrHolk6Tee8
An Error occured while reading.
Finished:  118QpWXmSgoaI
An Error occured while reading.
Finished:  11GUXJvrsI8LQ
Finished:  11vR8rsmrlx18
Finished:  9tdHQqqonvOk
Finished:  9uFKrD571OAc
Finished:  9QrWj_xTVFOs
Finished:  9CmjXlfkUls4
Finished:  9lnbEjlQ8KBk
Finished:  9cMMjQH8meuo
Finished:  4De2KvQ91QM4
Finished:  4kUZg8MPJjt4
Finished:  429uRE

Finished:  5WIS_7s__ND4
Finished:  5I9X0kKKEYwY
Finished:  5T6fyRCDerHk
Finished:  5AL9deoXg3NQ
Finished:  51iwdak9sW5A
Finished:  5oFy-3Bz31NE
Finished:  52sBvJ6-tXDc
Finished:  55cB7AF0VFJI
Finished:  50dyG75subpA
Finished:  5ijiMxeh_5Ck
Finished:  57_B857jMRnM
Finished:  56ihg-fUEyUI
Finished:  5XVOWF0O4UiE
Finished:  5VpxgG7NbxSM
Finished:  50bL8KT2OgLM
Finished:  5wCwlAy_ATX4
Finished:  51sqEMX0HcKQ
Finished:  5wIZQo8E3z3g
Finished:  5Y99gz3RZtpo
Finished:  5RsLxU7W8Eyc
Finished:  55LiPCu-meiA
Finished:  5ViicxSKZQR0
Finished:  5nZ01lT6OmVE
Finished:  5aMWLEaVQ0eI
Finished:  5fyWORJSYMF0
Finished:  5m0upE92JqQ4
Finished:  58ASDW9pNIyQ
Finished:  5OhQWRBMVoIA
Finished:  5c549Y9y98kM
Finished:  5huZW1LHjUG4
Finished:  5U5cS4RtVIM8
Finished:  5gRTh9r5cqs4
Finished:  5Kg04ilgZyPw
Finished:  5b5hPtQ655Hs
Finished:  5YhuIEFp_ro0
Finished:  5thvh-sN13pU
Finished:  5Jr_IjnbS4po
Finished:  5CE_OIelucKY
Finished:  5goTmlGVcAZ4
Finished:  37uHtzYkPsI0
Finished:  3M1h86MYoYO0
Finished:  110Ge

Finished:  5uXuLiL5bMjw
Finished:  4SY3eUU1DcHc
Finished:  4za1gPyZ99Dk
Finished:  4kOSe6ClmmwQ
Finished:  4Foepf3b5P58
Finished:  4NL5Xh18hf2c
Finished:  4uAIhp_Q4cHY
Finished:  4d6sTYAjwS3I
Finished:  8O3d6gs7qJVU
Finished:  7VXhLOkoseeU
Finished:  4hICFthdXYVA
Finished:  5lYemvvmcJsc
Finished:  4AFUjLULgn8E
Finished:  5DwF0AG2l36Q
Finished:  12qQElYQNDjTk
Finished:  11oQeXxKPSX58
Finished:  641Bu7vQOEVA
Finished:  6LPF2T8XAmbI
Finished:  7P91-1kX-a2M
Finished:  8PPXD8SNWceE
Finished:  11g3tVdd9wxGc
Finished:  12JwDawunBdj0
Finished:  12BYhR6OBQ4zA
Finished:  11MBMVdWxfwRk
Finished:  12c_km987s7HE
Finished:  8MUwpedyLOTg
Finished:  8X7SQAielcis
Finished:  9LwSaiTxjjpg
Finished:  9MMQtI6b_C6U
Finished:  5vfSqjtTx9QU
Finished:  3NA6hlZLq6FM
Finished:  4gf7zJ9d49vM
Finished:  79HehtqM6_tA
Finished:  5Nrkdepe5pms
Finished:  5_p_CY_QyMgs
Finished:  4OpO2Pmn2Ko4
Finished:  4ZvxZfY3ZY7M
Finished:  4GU6D4zezbjI
Finished:  4TAbj9kDkERE
Finished:  4gb_5ORc_6C8
Finished:  4LOBLmOdeODs
Finished: